In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import random
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
df = pd.read_excel("/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/data.xlsx",engine="openpyxl")
df.head()

# Extracting cataract information from the Dataset

In [ ]:
#making the target dataframes from the doctor's comments 
def has_cataract(text):
    if "cataract" == text:
        return 1
    else:
        return 0
df["left_cataract"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
df["right_cataract"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_cataract(x))


#making 2 arrays with the training dataset
left_cataract = df.loc[ (df.left_cataract == 1)]["Left-Fundus"].values
print(left_cataract[:10])
right_cataract = df.loc[ (df.right_cataract == 1)]["Right-Fundus"].values
print(right_cataract[:10])

In [ ]:
print("Number of images in left cataract: {}".format(len(left_cataract)))
print("Number of images in right cataract: {}".format(len(right_cataract)))

# Extracting mild nonproliferative retinopathy information from the Dataset

In [ ]:
#making the target dataframes from the doctor's comments 
def has_mild_nonproliferative_retinopathy(text):
    if "mild nonproliferative retinopathy" == text:
        return 1
    else:
        return 0
df["left_mnr"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_mild_nonproliferative_retinopathy(x))
df["right_mnr"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_mild_nonproliferative_retinopathy(x))

#making 2 arrays with the training dataset
left_mnr = df.loc[ (df.left_mnr == 1)]["Left-Fundus"].values
print(left_mnr[:10])
right_mnr = df.loc[ (df.right_mnr == 1)]["Right-Fundus"].values
print(right_mnr[:10])

In [ ]:
print("Number of images in left mild_nonproliferative_retinopathy: {}".format(len(left_mnr)))
print("Number of images in right mild_nonproliferative_retinopathy: {}".format(len(right_mnr)))

# pathological myopia

In [ ]:
#making the target dataframes from the doctor's comments 
def has_myopia(text):
    if "pathological myopia" == text:
        return 1
    else:
        return 0
df["left_myopia"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_myopia(x))
df["right_myopia"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_myopia(x))


#making 2 arrays with the training dataset
left_myopia = df.loc[ (df.left_myopia == 1)]["Left-Fundus"].values
print(left_myopia[:10])
right_myopia = df.loc[ (df.right_myopia == 1)]["Right-Fundus"].values
print(right_myopia[:10])

In [ ]:
print("Number of images in left pathological myopia: {}".format(len(left_myopia)))
print("Number of images in right pathological myopia: {}".format(len(right_myopia)))

# tessellated

In [ ]:
#making the target dataframes from the doctor's comments 
def has_tessellated(text):
    if "tessellated fundus，peripapillary atrophy" == text:
        return 1
    else:
        return 0
df["left_tessellated"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_tessellated(x))
df["right_tessellated"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_tessellated(x))


#making 2 arrays with the training dataset
left_tessellated = df.loc[ (df.left_tessellated == 1)]["Left-Fundus"].values
print(left_tessellated[:10])
right_tessellated = df.loc[ (df.left_tessellated == 1)]["Right-Fundus"].values
print(right_tessellated[:10])

In [ ]:
print("Number of images in tessellated: {}".format(len(left_tessellated)))
print("Number of images in tessellated: {}".format(len(right_tessellated)))

# wet

In [ ]:
#making the target dataframes from the doctor's comments 
def has_wet(text):
    if "wet age-related macular degeneration" == text:
        return 1
    else:
        return 0
df["left_wet"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_wet(x))
df["right_wet"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_wet(x))


#making 2 arrays with the training dataset
left_wet = df.loc[ (df.left_wet == 1)]["Left-Fundus"].values
print(left_wet[:10])
right_wet = df.loc[ (df.right_wet == 1)]["Right-Fundus"].values
print(right_wet[:10])

In [ ]:
print("Number of images in wet: {}".format(len(left_wet)))
print("Number of images in wet: {}".format(len(right_wet)))

In [ ]:
#concating two arrays to make an array for age-related macular degeneration dataset (pic file names) and normal file names
cataract = np.concatenate((left_cataract,right_cataract),axis=0)
mild_retinopathy = np.concatenate((left_mnr,right_mnr),axis=0)
wet = np.concatenate((left_wet,right_wet),axis=0)
myopia = np.concatenate((left_myopia,right_myopia),axis=0)
tessellated = np.concatenate((left_tessellated,right_tessellated),axis=0)

In [ ]:
print(len(cataract),len(mild_retinopathy),len(wet),len(myopia),len(tessellated))


# Creating Dataset from images

In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
dataset_dir = "/kaggle/input/myodir-crop/myodir-crop"
image_size=224 # will make a square 224*224 image. Taking this for the VGG model
labels = []
dataset = []
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(dataset_dir,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR) #load 
            image = cv2.resize(image,(image_size,image_size)) #resize
        except:
            continue # if I fail to load or resize, continue
        
        dataset.append([np.array(image),np.array(label)]) #making the Dataset as pic, label. 0 index is pic and 1 is label
    random.shuffle(dataset)
    return dataset
        

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import random
from tqdm import tqdm
dataset = []
def create_dataset_one_hot(image_category, label):
    datagen = ImageDataGenerator(
        horizontal_flip=True,  
       
    )
    
    for img in tqdm(image_category, disable=True):  # disable tqdm for this environment
        image_path = os.path.join(dataset_dir, img)
        try:
            
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
            image = cv2.resize(image, (image_size, image_size))  # 
            image = image / 255.0
            # Convert the label to one-hot encoding
            label_one_hot = to_categorical(label, num_classes=4)
            augmented_image = datagen.random_transform(image)
        
            dataset.append([np.array(augmented_image), label_one_hot])
        except:
            continue  # if I fail to load or resize, continue
        
          # making the Dataset as pic, label. 0 index is pic and 1 is label
    random.shuffle(dataset)
    return dataset


In [ ]:
dataset = create_dataset_one_hot(mild_retinopathy,0)

In [ ]:
len(dataset)

In [ ]:
dataset = create_dataset_one_hot(cataract,1)

In [ ]:
len(dataset)

In [ ]:
dataset = create_dataset_one_hot(myopia,2)
len(dataset)

In [ ]:
#dataset = create_dataset_one_hot(tessellated,3)

len(dataset)

In [ ]:
dataset = create_dataset_one_hot(wet,3)
len(dataset)

In [ ]:
plt.figure(figsize=(12,7))
data_category = ["mild nonproliferative retinopathy","cataract","pathological myopia","wet degeneration macular"]
for i in range(10):
    sample = random.choice(range(len(dataset)))
    image = dataset[sample][0]
    print(dataset[sample][1])
    
    class_index = np.argmax(dataset[sample][1])
    label = data_category[class_index]
   
    plt.subplot(2,5,i+1)
    plt.imshow(image)
    plt.xlabel(label)
plt.tight_layout()    

# Dividing dataset into x(features) & y(target)

In [ ]:
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

Train Test Split (80/20)

In [ ]:
from sklearn.model_selection import train_test_split


x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3)


x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5)


In [ ]:
print(len(x_train))
print(len(x_val))
print(len(x_test))

# Creating Model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model_resnet = InceptionV3(
    weights="imagenet",
    include_top=False, #i want to fit my images as input, not their input
    input_shape=(224,224,3) #224*224 size, 3 RGB channel 
)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def create_model(vgg):
    model = Sequential() # will add layers sequentially, more info: https://keras.io/api/
    model.add(vgg)
    model.add(Dense(256, activation='relu'))
    model.add(layers.Dropout(rate=0.5))
    model.add(Dense(128, activation='sigmoid'))
    model.add(layers.Dropout(rate=0.3))
    


    model.add(Flatten()) #linearizing the images. output of previous layer of size 224*224*3 is flattened to 1 dimension
    model.add(Dense(4,activation="softmax")) #should use sigmoid in final layer, takes the max val in calc, good for final layer/decision
    return model


In [ ]:
"""def create_model(resnet):
    x = resnet.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(5, activation='sigmoid')(x)
    model = Model(inputs=resnet.input, outputs=predictions)
    return model"""


In [ ]:
model = create_model(model_resnet)







In [ ]:
from tensorflow.keras.optimizers import Adam

# Configura el learning rate deseado
learning_rate = 0.0001  # Puedes ajustar este valor según tus necesidades

# Crea una instancia del optimizador Adam con tu learning rate personalizado
adam_optimizer = Adam(learning_rate=learning_rate)

# Compila tu modelo con el optimizador Adam personalizado
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#adam is a back propagator optimizer, loss function & metrics are performance values that I want

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
y_train_labels = np.argmax(y_train,axis=1)
# Aquí asumimos que y_train_labels es un arreglo de etiquetas para tus datos de entrenamiento
class_weights = compute_sample_weight(class_weight='balanced', y=y_train_labels)

# Ahora class_weights es un arreglo de pesos, uno por cada muestra en y_train_labels
# Si necesitas un diccionario de pesos por clase, puedes hacer lo siguiente:
unique_classes = np.unique(y_train_labels)
cls_wt_dict = {cls: np.mean(class_weights[y_train_labels == cls]) for cls in unique_classes}

print(cls_wt_dict)


In [ ]:
print(len(x_train))
history = model.fit(x_train,y_train,batch_size=16,epochs=20,validation_data=(x_val,y_val),class_weight=cls_wt_dict)

In [ ]:
loss,accuracy = model.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Asumimos que 'history' es el objeto retornado por el método 'fit' del modelo
# y que detuviste el entrenamiento en la época 4.

# Visualizar la función de pérdida y precisión durante el entrenamiento y validación
plt.figure(figsize=(14, 5))

# Subplot para la función de pérdida
plt.subplot(1, 2, 1)
# Asegúrate de incluir solo las épocas hasta las que entrenaste
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Evolution')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Subplot para la precisión
plt.subplot(1, 2, 2)
# Asegúrate de incluir solo las épocas hasta las que entrenaste
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Evolution')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()




In [ ]:
print()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_pred_probs = model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)




In [ ]:
print(classification_report(y_true_classes, y_pred_classes))

# Making Actual Predictions using the Model

In [ ]:


plt.figure(figsize=(12,7))
for i in range(10):
    sample = random.choice(range(len(x_test)))
    image = x_test[sample]
    category = y_true_classes[sample]
    pred_category = y_pred_classes[sample]
    
    label = data_category[category]
    pred_label = data_category[pred_category]
        
    plt.subplot(2,5,i+1)
    plt.imshow(image)
    plt.xlabel("Actual:{}\nPrediction:{}".format(label,pred_label))
plt.tight_layout()

In [ ]:
clf=model
p=0
q=1
prediction_train = clf.predict(x_train)
prediction_test = clf.predict(x_test)
# prediction_train = [1 if x>0.5 else 0 for x in prediction_train]
for i in range(len(prediction_train)):
    if(prediction_train[i][0] > 0.3):
        prediction_train[i][0] = 1
    else:
        prediction_train[i][0] = 0

for i in range(len(prediction_test)):
    if(prediction_test[i][0] > 0.3):
        prediction_test[i][0] = 1
    else:
        prediction_test[i][0] = 0
  

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

# Suponiendo que 'model' es tu modelo entrenado y que 'x_train' y 'x_test' son tus datos
prediction_train = model.predict(x_train)
prediction_test = model.predict(x_test)

# Convertir predicciones de one-hot a etiquetas
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
prediction_train_labels = np.argmax(prediction_train, axis=1)
prediction_test_labels = np.argmax(prediction_test, axis=1)

# Calcula las métricas
acc_train = accuracy_score(y_train_labels, prediction_train_labels)
precision_train = precision_score(y_train_labels, prediction_train_labels, average='macro')
recall_train = recall_score(y_train_labels, prediction_train_labels, average='macro')
f1_train = f1_score(y_train_labels, prediction_train_labels, average='macro')

acc_test = accuracy_score(y_test_labels, prediction_test_labels)
precision_test = precision_score(y_test_labels, prediction_test_labels, average='macro')
recall_test = recall_score(y_test_labels, prediction_test_labels, average='macro')
f1_test = f1_score(y_test_labels, prediction_test_labels, average='macro')

# No se puede calcular especificidad directamente para multiclase en sklearn, tendrías que hacerlo manualmente o buscar otra librería

# Añade los resultados a tu DataFrame
evaluation = pd.DataFrame({'Model': ['Neural Network'],
                           'Accuracy(train)': [acc_train],
                           'Precision(train)': [precision_train],
                           'Recall(train)': [recall_train],
                           'F1_score(train)': [f1_train],
                           'Specificity(train)': [np.nan],  # No calculado
                           'Accuracy(test)': [acc_test],
                           'Precision(test)': [precision_test],
                           'Recalll(test)': [recall_test],
                           'F1_score(test)': [f1_test],
                           'Specificity(test)': [np.nan],  # No calculado
                          })

print(evaluation.sort_values(by='Accuracy(test)', ascending=False))


In [ ]:
print(len(y_pred_classes))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Matriz de confusión
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Calcula la curva ROC y AUC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = 4

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Visualizar las curvas ROC para cada clase
plt.figure(figsize=(10, 8))
colors = cycle(['blue', 'red', 'green', 'yellow', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC and AUC for multi-class')
plt.legend(loc="lower right")
plt.show()


In [ ]:
model.save("modelInception4clases.h5")
model.save_weights("modelInception4clasesweights.h5")

In [ ]:
from IPython.display import FileLink
import os

# Reemplaza 'tu_archivo.csv' con el nombre de tu archivo
archivo = '/kaggle/working/modelInception4clases.h5'

# Verifica si el archivo existe
if os.path.exists(archivo):
    # Crea un enlace de descarga
    display(FileLink(archivo))
else:
    print("no existe")